# Data Cleaning

Section goals:
- clean data
- store as cleaned csv files

API and data dictionary:
https://steamspy.com/api.php

Future ideas:
- pc requirements analysis over time


In [2]:
# load extensions and magics

# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

# http://raw.github.com/jrjohansson/version_information/master/version_information.py
%load_ext version_information
%version_information numpy, pandas

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information


Software versions
Python 3.7.3 64bit [MSC v.1915 64 bit (AMD64)]
IPython 7.5.0
OS Windows 10 10.0.17763 SP0
numpy 1.16.3
pandas 0.24.2
Thu May 02 11:52:22 2019 GMT Summer Time

In [3]:
from ast import literal_eval

import pandas as pd

In [48]:
# customisations
pd.options.display.max_columns = 100

In [50]:
raw_steam_data = pd.read_csv('../data/steam_app_data.csv')

print('Rows:', raw_steam_data.shape[0])
print('Columns:', raw_steam_data.shape[1])
raw_steam_data.head()

Rows: 29235
Columns: 39


,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,game,Counter-Strike,10,0.0,False,NaN,NaN,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,NaN,"English<strong>*</strong>, French<strong>*</st...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'GBP', 'initial': 719, 'final': 7...",[7],"[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","{'score': 88, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 65735},{'total': 0},"{'coming_soon': False, 'date': '1 Nov, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
1,game,Team Fortress Classic,20,0.0,False,NaN,NaN,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,NaN,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'GBP', 'initial': 399, 'final': 3...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 2802},{'total': 0},"{'coming_soon': False, 'date': '1 Apr, 1999'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
2,game,Day of Defeat,30,0.0,False,NaN,NaN,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,NaN,"English, French, German, Italian, Spanish - Spain",https://steamcdn-a.akamaihd.net/steam/apps/30/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'GBP', 'initial': 399, 'final': 3...",[30],"[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","{'score': 79, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 1992},{'total': 0},"{'coming_soon': False, 'date': '1 May, 2003'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/30/...,"{'ids': [], 'notes': None}"
3,game,Deathmatch Classic,40,0.0,False,NaN,NaN,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,NaN,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</s

In [442]:
def process_null_cols(df, thresh=0.5):
    cutoff_count = len(df) * thresh

    null_counts = df.isnull().sum()
    null_columns = df.columns[null_counts > cutoff_count]
    # print('Dropped columns:', list(null_columns))

    return df.dropna(thresh=cutoff_count, axis=1)


def process_null_row(df, col):
    null_rows = df[df[col].isnull()].shape[0]
    # print("Dropped null '{}' rows:".format(col), null_rows)
    
    return df[df[col].notnull()]
    
    
def process_name(df):
    df = process_null_row(df, 'name')
    # print("Dropped 'none' name rows:", df[df['name'] == 'none'].shape[0])
    return df[df['name'] != 'none']


def process_age(df):
    # UK PEGI Age raings: 3, 7, 12, 16, 18
    cut_points = [-1, 0, 3, 7, 12, 16, 2000]
    label_values = [0, 3, 7, 12, 16, 18]
    
    df['required_age'] = pd.cut(df['required_age'], cut_points, labels=label_values)
    return df


def process_release_date(df):
    df = df.copy()
    df['coming_soon'] = df['release_date'].apply(lambda x: literal_eval(x)['coming_soon'])
    # Only want released games
    df = df[df['coming_soon'] == False].copy()
    
    # probably more efficient way to handle this, but works for now
    df['date'] = df['release_date'].apply(lambda x: literal_eval(x)['date'])
    df.loc[df['date'] == '', 'date'] = None
    df['datetime'] = pd.to_datetime(df['date'], format='%d %b, %Y', errors='ignore')
    df['datetime'] = pd.to_datetime(df['date'], format='%b %Y', errors='ignore')
    df['release_date'] = pd.to_datetime(df['datetime'])
    
    df = df.drop(['coming_soon', 'date', 'datetime'], axis=1)
    return df


def convert_price(x):
    try:
        return literal_eval(x)
    except ValueError:
        return {'currency': 'GBP', 'initial': -1}

    
def process_price(df):
    df = df.copy()
    
    # evaluate as dictionary and set to -1 if missing
    df['price_overview'] = df['price_overview'].apply(convert_price)
    
    df['currency'] = df['price_overview'].apply(lambda x: x['currency'])
    df['price'] = df['price_overview'].apply(lambda x: x['initial'])
    
    # Set free games to 0
    df.loc[df['is_free'], 'price'] = 0
    # 4 apps in USD or EUR. Set to -1 for now
    df.loc[df['currency'] != 'GBP', 'price'] = -1
    
    df = df.drop(['is_free', 'currency', 'price_overview'], axis=1)
    
    # store price in pounds, not pennies
    df['price'] /= 100
    
    return df


def is_english(x):
    if 'english' in x.lower():
        return True
    else:
        return False

    
def process_language(df):
    df = df.copy()
    
    # 13 rows with missing data, safe to drop
    df = df.dropna(subset=['supported_languages'])
    
    df['english'] = df['supported_languages'].apply(is_english)
    df = df.drop('supported_languages', axis=1)
    
    return df


steam_data = process_null_cols(raw_steam_data)
steam_data = process_null_row(steam_data, 'type')
steam_data = process_name(steam_data)
steam_data = steam_data.drop('type', axis=1)
steam_data = steam_data.drop_duplicates()
steam_data = process_age(steam_data)
steam_data = process_release_date(steam_data)
steam_data = process_price(steam_data)

# could be useful for future recommender or key-word analysis, but drop for now
steam_data = steam_data.drop(['detailed_description', 'about_the_game', 'short_description'], axis=1)

steam_data = process_language(steam_data)

# links to images. could be useful for future image analysis project
steam_data = steam_data.drop(['header_image', 'screenshots', 'background'], axis=1)

# nearly 10000 missing values, unlikely useful to this analysis
steam_data = steam_data.drop('website', axis=1)

steam_data.head()

,name,steam_appid,required_age,pc_requirements,mac_requirements,linux_requirements,developers,publishers,packages,package_groups,platforms,categories,genres,movies,achievements,release_date,support_info,content_descriptors,price,english
0,Counter-Strike,10,0,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[7],"[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},2000-11-01,"{'url': 'http://steamcommunity.com/app/10', 'e...","{'ids': [2, 5], 'notes': 'Includes intense vio...",7.19,True
1,Team Fortress Classic,20,0,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},1999-04-01,"{'url': '', 'email': ''}","{'ids': [2, 5], 'notes': 'Includes intense vio...",3.99,True
2,Day of Defeat,30,0,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[30],"[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},2003-05-01,"{'url': '', 'email': ''}","{'ids': [], 'notes': None}",3.99,True
3,Deathmatch Classic,40,0,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[31],"[{'name': 'default', 'title': 'Buy Deathmatch ...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},2001-06-01,"{'url': '', 'email': ''}","{'ids': [], 'notes': None}",3.99,True
4,Half-Life: Opposing Force,50,0,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Gearbox Software'],['Valve'],[32],"[{'name': 'default', 'title': 'Buy Half-Life: ...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},1999-11-01,"{'url': 'https://help.steampowered.com', 'emai...","{'ids': [], 'notes': None}",3.99,True


In [451]:
steam_data['pc_requirements']

cols = ['name', 'pc_requirements', 'mac_requirements', 'linux_requirements']

for i in [0, 10, 100, 1000, 10000]:
    for col, val in zip(cols, steam_data.iloc[i][cols]):
        print('-'*20 + col + '-'*(30-len(col)) + '\n')
        try:
            display(literal_eval(val), '\n')
        except:
            display(val, '\n')

--------------------name--------------------------



'Counter-Strike'

'\n'

--------------------pc_requirements---------------



{'minimum': '\r\n\t\t\t<p><strong>Minimum:</strong> 500 mhz processor, 96mb ram, 16mb video card, Windows XP, Mouse, Keyboard, Internet Connection<br /></p>\r\n\t\t\t<p><strong>Recommended:</strong> 800 mhz processor, 128mb ram, 32mb+ video card, Windows XP, Mouse, Keyboard, Internet Connection<br /></p>\r\n\t\t\t'}

...

[]

'\n'

--------------------linux_requirements------------



[]

'\n'

In [443]:
steam_data.isnull().sum()

name                      0
steam_appid               0
required_age              0
pc_requirements           0
mac_requirements          0
linux_requirements        0
developers              112
publishers                0
packages               2919
package_groups            0
platforms                 0
categories              553
genres                   40
movies                 1883
achievements           1935
release_date             56
support_info              0
content_descriptors       0
price                     0
english                   0
dtype: int64

In [434]:
def is_english(x):
    if 'english' in x.lower():
        return True
    else:
        return False

def process_language(df):
    df = df.copy()
    
    # 13 rows with missing data, safe to drop
    df = df.dropna(subset=['supported_languages'])
    
    df['english'] = df['supported_languages'].apply(is_english)
    df = df.drop('supported_languages', axis=1)
    
    return df


temp_df = process_language(steam_data)
print(temp_df.shape[0])
temp_df.head()
# temp_df[['english', 'supported_languages']].head()
# temp_df['english'].value_counts()

# temp_df[temp_df['english'] == False]
# temp_df[temp_df['english'] == False]['supported_languages'].value_counts()

# temp_df['supported_languages'].isnull().sum()
# 13 null apps, safe to drop
# temp_df[temp_df['supported_languages'].isnull()]

28765


,name,steam_appid,required_age,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,packages,package_groups,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors,price,english
0,Counter-Strike,10,0,https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[7],"[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},2000-11-01,"{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"{'ids': [2, 5], 'notes': 'Includes intense vio...",7.19,True
1,Team Fortress Classic,20,0,https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},1999-04-01,"{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,"{'ids': [2, 5], 'notes': 'Includes intense vio...",3.99,True
2,Day of Defeat,30,0,https://steamcdn-a.akamaihd.net/steam/apps/30/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[30],"[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},2003-05-01,"{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/30/...,"{'ids': [], 'notes': None}",3.99,True
3,Deathmatch Classic,40,0,https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[31],"[{'name': 'default', 'title': 'Buy Deathmatch ...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},2001-06-01,"{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/40/...,"{'ids': [], 'notes': None}",3.99,True
4,Half-Life: Opposing Force,50,0,https://steamcdn-a.akamaihd.net/steam/apps/50/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Gearbox Software'],['Valve'],[32],"[{'name': 'default', 'title': 'Buy Half-Life: ...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},1999-11-01,"{'url': 'https://help.steampowered.com', 'emai...",https://steamcdn-a.akamaihd.net/steam/apps/50/...,"{'ids': [], 'notes': None}",3.99,True


In [408]:
cols = ['name', 'detailed_description', 'about_the_game', 'short_description']

for i in [0, 10, 100, 1000, 10000]:
    for col, val in zip(cols, steam_data.iloc[i][cols]):
        print('-'*20 + col + '-'*(30-len(col)) + '\n')
        # print(val, '\n')
        
steam_data.drop(['detailed_description', 'about_the_game', 'short_description'], axis=1)

--------------------name--------------------------

--------------------detailed_description----------

--------------------about_the_game----------------

--------------------short_description-------------

--------------------name--------------------------

--------------------detailed_description----------

--------------------about_the_game----------------

--------------------short_description-------------

--------------------name--------------------------

--------------------detailed_description----------

--------------------about_the_game----------------

--------------------short_description-------------

--------------------name--------------------------

--------------------detailed_description----------

--------------------about_the_game----------------

--------------------short_description-------------

--------------------name--------------------------

--------------------detailed_description----------

--------------------about_the_game----------------

------------

,name,steam_appid,required_age,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,packages,package_groups,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors,price
0,Counter-Strike,10,0,"English<strong>*</strong>, French<strong>*</st...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[7],"[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},2000-11-01,"{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"{'ids': [2, 5], 'notes': 'Includes intense vio...",7.19
1,Team Fortress Classic,20,0,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},1999-04-01,"{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,"{'ids': [2, 5], 'notes': 'Includes intense vio...",3.99
2,Day of Defeat,30,0,"English, French, German, Italian, Spanish - Spain",https://steamcdn-a.akamaihd.net/steam/apps/30/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[30],"[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},2003-05-01,"{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/30/...,"{'ids': [], 'notes': None}",3.99
3,Deathmatch Classic,40,0,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[31],"[{'name': 'default', 'title': 'Buy Deathmatch ...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},2001-06-01,"{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/40/...,"{'ids': [], 'notes': None}",3.99
4,Half-Life: Opposing Force,50,0,"English, French, German, Korean",https://steamcdn-a.akamaihd.net/steam/apps/50/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Gearbox Software'],['Valve'],[32],"[{'name': 'default', 'title': 'Buy Half-Life: ...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},1999-11-01,"{'url': 'https://help.steampowered.com', 'emai...",https://steamcdn-a.akamaihd.net/steam/apps/50/...,"{'ids': [], 'notes': None}",3.99
5,Ricochet,60,0,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/60/...,NaN,{'minimum': 

In [394]:
def convert_price(x):
    try:
        return literal_eval(x)
    except ValueError:
        return {'currency': 'GBP', 'initial': -1}

def process_price(df):
    df = df.copy()
    
    # evaluate as dictionary and set to -1 if missing
    df['price_overview'] = df['price_overview'].apply(convert_price)
    
    df['currency'] = df['price_overview'].apply(lambda x: x['currency'])
    df['price'] = df['price_overview'].apply(lambda x: x['initial'])
    
    # Set free games to 0
    df.loc[df['is_free'], 'price'] = 0
    # 4 apps in USD or EUR. Set to -1 for now
    df.loc[df['currency'] != 'GBP', 'price'] = -1
    
    df = df.drop(['is_free', 'currency', 'price_overview'], axis=1)
    
    df['price'] /= 100
    
    return df


temp_df = process_price(steam_data)
temp_df.head(20)
# temp_df[temp_df['currency'] != 'GBP']

,name,steam_appid,required_age,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,packages,package_groups,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors,price
0,Counter-Strike,10,0,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,"English<strong>*</strong>, French<strong>*</st...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[7],"[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},2000-11-01,"{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"{'ids': [2, 5], 'notes': 'Includes intense vio...",7.19
1,Team Fortress Classic,20,0,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},1999-04-01,"{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,"{'ids': [2, 5], 'notes': 'Includes intense vio...",3.99
2,Day of Defeat,30,0,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,"English, French, German, Italian, Spanish - Spain",https://steamcdn-a.akamaihd.net/steam/apps/30/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[30],"[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},2003-05-01,"{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/30/...,"{'ids': [], 'notes': None}",3.99
3,Deathmatch Classic,40,0,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[31],"[{'name': 'default', 'title': 'Buy Deathmatch ...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},2001-06-01,"{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/40/...,"{'ids': [], 'notes': None}",3.99
4,Half-Life: Opposing Force,50,0,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,"English, Fren

In [368]:
steam_data['price_overview'][0]

"{'currency': 'GBP', 'initial': 719, 'final': 719, 'discount_percent': 0, 'initial_formatted': '', 'final_formatted': '£7.19'}"

In [351]:
from ast import literal_eval

def process_release_date(df):
    df = df.copy()
    df['coming_soon'] = df['release_date'].apply(lambda x: literal_eval(x)['coming_soon'])
    # Only want released games
    df = df[df['coming_soon'] == False].copy()
    
    df['date'] = df['release_date'].apply(lambda x: literal_eval(x)['date'])
    df.loc[df['date'] == '', 'date'] = None
    df['datetime'] = pd.to_datetime(df['date'], format='%d %b, %Y', errors='ignore')
    df['datetime'] = pd.to_datetime(df['date'], format='%b %Y', errors='ignore')
    df['release_date'] = pd.to_datetime(df['datetime'])
    
    df = df.drop(['coming_soon', 'date', 'datetime'], axis=1)
    return df

temp_df = process_release_date(steam_data)

temp_df['release_date'].value_counts(dropna=False)
# temp_df
# date_filter = (temp_df['date'].notnull()) & (temp_df['datetime'].isnull())
# # temp_df[date_filter]['datetime'] = pd.to_datetime(temp_df[date_filter]['date'], format='%b %Y')
# temp_df[temp_df['steam_appid'] == 29160]
# temp_df[temp_df['datetime'].isnull()]

# temp_df[temp_df['name'].isnull()]
# temp_df.iloc[245:255]
# temp_df.dtypes
# temp_df.head()
# temp_df[temp_df['date'] == '']
# temp_df[['release_date', 'coming_soon', 'date']]

2018-07-13    65
NaT           64
2019-01-31    59
2016-04-05    59
2017-12-01    57
2018-11-16    56
2018-05-17    56
2018-09-14    56
2018-05-31    55
2018-06-15    54
2018-11-09    53
2018-12-20    52
2019-02-01    52
2019-02-28    52
2018-11-30    51
2017-09-15    51
2018-03-08    51
2019-04-18    51
2018-12-14    51
2019-02-22    51
2018-04-27    50
2017-10-31    49
2018-08-17    49
2018-04-13    49
2018-04-06    48
2018-02-08    48
2018-09-28    48
2018-11-15    47
2017-09-14    47
2019-03-28    47
              ..
2006-10-05     1
2016-04-24     1
2012-06-21     1
2008-03-07     1
2011-08-10     1
2010-11-17     1
2001-12-01     1
2016-03-06     1
2011-12-02     1
2016-10-08     1
2009-07-24     1
2017-03-19     1
2008-06-24     1
2013-11-29     1
2009-04-14     1
2013-05-16     1
2017-12-09     1
2009-06-04     1
2015-12-06     1
2009-07-27     1
2012-06-07     1
2014-09-10     1
2011-02-02     1
2016-05-01     1
2011-06-07     1
2010-03-23     1
2012-07-04     1
2012-02-27    

In [197]:
# UK PEGI Age raings: 3, 7, 12, 16, 18
print(steam_data['required_age'].value_counts(dropna=False).sort_index())

def process_age(df):
    # UK PEGI Age raings: 3, 7, 12, 16, 18
    cut_points = [-1, 0, 3, 7, 12, 16, 2000]
    label_values = [0, 3, 7, 12, 16, 18]
    
    df['required_age'] = pd.cut(df['required_age'], cut_points, labels=label_values)
    return df

steam_data[(steam_data['required_age'] > 0) & (steam_data['required_age'] <= 3)]

steam_data = process_age(steam_data)
steam_data['required_age'].value_counts().sort_index()
# steam_data[steam_data['required_age'] > 0]

0     28431
3        11
7        12
12       79
16      205
18      337
Name: required_age, dtype: int64


0     28431
3        11
7        12
12       79
16      205
18      337
Name: required_age, dtype: int64

In [169]:
pd.concat(g for _, g in steam_data.groupby("name") if len(g) > 1)

,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors
26696,2048,942050,0.0,False,2048 is a classic puzzle with a simple goal; r...,2048 is a classic puzzle with a simple goal; r...,THE ORIGINAL MADE BETTER | 2048 is a classic p...,English,https://steamcdn-a.akamaihd.net/steam/apps/942...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,"['Lucas Alexander Visual Arts', 'Nicolai Clemm...",['Lucas Alexander Visual Arts'],"{'currency': 'GBP', 'initial': 449, 'final': 4...",[306041],"[{'name': 'default', 'title': 'Buy 2048', 'des...","{'windows': True, 'mac': False, 'linux': True}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://steamcdn...","[{'id': 256733767, 'name': 'Release Trailer', ...","{'total': 15, 'highlighted': [{'name': 'Just G...","{'coming_soon': False, 'date': '31 Oct, 2018'}","{'url': 'http://lucasalexander.org', 'email': ...",https://steamcdn-a.akamaihd.net/steam/apps/942...,"{'ids': [], 'notes': None}"
28734,2048,1026330,0.0,False,"2048 is played on a 4×4 grid, with numbered ti...","2048 is played on a 4×4 grid, with numbered ti...","2048 is a game with a mathematical component, ...",English<strong>*</strong><br><strong>*</strong...,https://steamcdn-a.akamaihd.net/steam/apps/102...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],['Dexion Games'],['Dexion Games'],"{'currency': 'GBP', 'initial': 79, 'final': 39...",[340990],"[{'name': 'default', 'title': 'Buy 2048', 'des...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://steamcdn...","[{'id': 256742064, 'name': 'Teaser', 'thumbnai...",NaN,"{'coming_soon': False, 'date': '21 Feb, 2019'}","{'url': '', 'email': 'gamesdexion@gmail.com'}",https://steamcdn-a.akamaihd.net/steam/apps/102...,"{'ids': [], 'notes': None}"
23302,Alone,837930,0.0,False,"<h2 class=""bb_tag""><strong>The Sun Falls Behin...","<h2 class=""bb_tag""><strong>The Sun Falls Behin...",Alone is a Pixelated Survival Game set in the ...,English<strong>*</strong><br><strong>*</strong...,https://steamcdn-a.akamaihd.net/steam/apps/837...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],['Killed Pixel Games'],['Killed Pixel Games'],"{'currency': 'GBP', 'initial': 79, 'final': 79...",[261295],"[{'name': 'default', 'title': 'Buy Alone', 'de...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '25', 'description': 'Adventure'}, {'i...","[{'id': 0, 'path_thumbnail': 'https://steamcdn...","[{'id': 256712913, 'name': 'Alone - the Only S...","{'total': 20, 'highlighted': [{'name': 'Frozen...","{'coming_soon': False, 'date': '1 May, 2018'}",{'url': 'https://killedpixel.wixsite.com/home'...,https://steamcdn-a.akamaihd.net/steam/apps/837...,"{'ids': [], 'notes': None}"
24568,Alone,871870,0.0,False,You wake up finding yourself alone in your cam...,You wake up finding yourself alone in your cam...,Alone is a 3D Puzzle Sidescroller with heavy f...,English<strong>*</strong><br><strong>*</strong...,https://steamcdn-a.akamaihd.net/steam/apps/871...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,['Avasion'],['Avasion'],"{'currency': 'GBP', 'initial': 79, 'final': 79...",[275553],"[{'name': 'default', 'title': 'Buy Alone', 'de...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'descriptio

In [159]:
steam_data[steam_data.duplicated(subset='steam_appid')]

,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors
31,SiN Episodes: Emergence,1300,0.0,False,"You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","English, Russian, French",https://steamcdn-a.akamaihd.net/steam/apps/130...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,[],[],['Ritual Entertainment'],['Ritual Entertainment'],"{'currency': 'GBP', 'initial': 719, 'final': 7...",[70],"[{'name': 'default', 'title': 'Buy SiN Episode...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '10 May, 2006'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/130...,"{'ids': [], 'notes': None}"
32,SiN Episodes: Emergence,1300,0.0,False,"You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","English, Russian, French",https://steamcdn-a.akamaihd.net/steam/apps/130...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,[],[],['Ritual Entertainment'],['Ritual Entertainment'],"{'currency': 'GBP', 'initial': 719, 'final': 7...",[70],"[{'name': 'default', 'title': 'Buy SiN Episode...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '10 May, 2006'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/130...,"{'ids': [], 'notes': None}"
356,Jagged Alliance 2 Gold,1620,0.0,False,<p>The small country of Arulco has been taken ...,<p>The small country of Arulco has been taken ...,The small country of Arulco has been taken ove...,English,https://steamcdn-a.akamaihd.net/steam/apps/162...,http://www.jaggedalliance2.com/,{'minimum': '<p><strong>Minimum Configuration:...,[],[],['Strategy First'],['Strategy First'],"{'currency': 'GBP', 'initial': 1499, 'final': ...",[94],"[{'name': 'default', 'title': 'Buy Jagged Alli...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '2', 'description': 'Strategy'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '6 Jul, 2006'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/162...,"{'ids': [], 'notes': None}"
493,Crazy Machines 1.5,18430,0.0,False,Create Your Own Zany &quot;Rube Goldberg&quot;...,Create Your Own Zany &quot;Rube Goldberg&quot;...,Create Your Own Zany &quot;Rube Goldberg&quot;...,English,https://steamcdn-a.akamaihd.net/steam/apps/184...,NaN,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],[],['Fakt Software'],['Viva Media'],"{'currency': 'GBP', 'initial': 699, 'final': 6...","[1242, 58401]","[{'name': 'default', 'title': 'Buy Crazy Machi...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '12 Dec, 2008'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/184...,"{'ids': [], 'notes': None}"
494,Crazy Machines 1.5,18430,0.0,False,Create Your Own Zany &quot;Rube Goldberg&quot;...,Create Your Own Zany &quot;Rube Goldberg&quot;...,Create Your Own Zany &quot;Rube Goldberg&quot;...,English,https://steamcdn-a.akamaihd.net/st

In [160]:
steam_data[steam_data['name'] == 'F.E.A.R.']

,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors
528,F.E.A.R.,21090,18.0,False,Be the hero in your own cinematic epic of acti...,Be the hero in your own cinematic epic of acti...,Experience the original F.E.A.R. along with F....,English,https://steamcdn-a.akamaihd.net/steam/apps/210...,https://wbgamessupport.wbgames.com/hc/en-us,{'minimum': '<strong>Minimum:</strong><br>\t\t...,[],[],"['Monolith Productions, Inc.', 'Timegate']",['Warner Bros. Interactive Entertainment'],"{'currency': 'GBP', 'initial': 3596, 'final': ...",[12357],"[{'name': 'default', 'title': 'Buy F.E.A.R.', ...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '21 May, 2010'}",{'url': 'https://wbgamessupport.wbgames.com/hc...,https://steamcdn-a.akamaihd.net/steam/apps/210...,"{'ids': [], 'notes': None}"
530,F.E.A.R.,21090,18.0,False,Be the hero in your own cinematic epic of acti...,Be the hero in your own cinematic epic of acti...,Experience the original F.E.A.R. along with F....,English,https://steamcdn-a.akamaihd.net/steam/apps/210...,https://wbgamessupport.wbgames.com/hc/en-us,{'minimum': '<strong>Minimum:</strong><br>\t\t...,[],[],"['Monolith Productions, Inc.', 'Timegate']",['Warner Bros. Interactive Entertainment'],"{'currency': 'GBP', 'initial': 3596, 'final': ...",[12357],"[{'name': 'default', 'title': 'Buy F.E.A.R.', ...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '21 May, 2010'}",{'url': 'https://wbgamessupport.wbgames.com/hc...,https://steamcdn-a.akamaihd.net/steam/apps/210...,"{'ids': [], 'notes': None}"
531,F.E.A.R.,21090,18.0,False,Be the hero in your own cinematic epic of acti...,Be the hero in your own cinematic epic of acti...,Experience the original F.E.A.R. along with F....,English,https://steamcdn-a.akamaihd.net/steam/apps/210...,https://wbgamessupport.wbgames.com/hc/en-us,{'minimum': '<strong>Minimum:</strong><br>\t\t...,[],[],"['Monolith Productions, Inc.', 'Timegate']",['Warner Bros. Interactive Entertainment'],"{'currency': 'GBP', 'initial': 3596, 'final': ...",[12357],"[{'name': 'default', 'title': 'Buy F.E.A.R.', ...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '21 May, 2010'}",{'url': 'https://wbgamessupport.wbgames.com/hc...,https://steamcdn-a.akamaihd.net/steam/apps/210...,"{'ids': [], 'notes': None}"


In [125]:
steam_data['name'].value_counts(dropna=False)

none                                                   3
F.E.A.R.                                               3
Dark Matter                                            3
SiN Episodes: Emergence                                3
Crazy Machines 1.5                                     3
Luna                                                   2
Dystopia                                               2
Aurora                                                 2
Cortex                                                 2
Evolution                                              2
The Great Escape                                       2
City Builder                                           2
Dream                                                  2
Prisoner                                               2
Alter Ego                                              2
Arena                                                  2
Don't Look Back                                        2
Beyond the Wall                

In [148]:
# steam_data[steam_data['steam_appid'] in steam_data['steam_appid'].value_counts()[steam_data['steam_appid'].value_counts() > 1].index.tolist()]

pd.concat(g for _, g in steam_data.groupby("steam_appid") if len(g) > 1)

,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors
30,SiN Episodes: Emergence,1300,0.0,False,"You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","English, Russian, French",https://steamcdn-a.akamaihd.net/steam/apps/130...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,[],[],['Ritual Entertainment'],['Ritual Entertainment'],"{'currency': 'GBP', 'initial': 719, 'final': 7...",[70],"[{'name': 'default', 'title': 'Buy SiN Episode...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '10 May, 2006'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/130...,"{'ids': [], 'notes': None}"
31,SiN Episodes: Emergence,1300,0.0,False,"You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","English, Russian, French",https://steamcdn-a.akamaihd.net/steam/apps/130...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,[],[],['Ritual Entertainment'],['Ritual Entertainment'],"{'currency': 'GBP', 'initial': 719, 'final': 7...",[70],"[{'name': 'default', 'title': 'Buy SiN Episode...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '10 May, 2006'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/130...,"{'ids': [], 'notes': None}"
32,SiN Episodes: Emergence,1300,0.0,False,"You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","English, Russian, French",https://steamcdn-a.akamaihd.net/steam/apps/130...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,[],[],['Ritual Entertainment'],['Ritual Entertainment'],"{'currency': 'GBP', 'initial': 719, 'final': 7...",[70],"[{'name': 'default', 'title': 'Buy SiN Episode...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '10 May, 2006'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/130...,"{'ids': [], 'notes': None}"
355,Jagged Alliance 2 Gold,1620,0.0,False,<p>The small country of Arulco has been taken ...,<p>The small country of Arulco has been taken ...,The small country of Arulco has been taken ove...,English,https://steamcdn-a.akamaihd.net/steam/apps/162...,http://www.jaggedalliance2.com/,{'minimum': '<p><strong>Minimum Configuration:...,[],[],['Strategy First'],['Strategy First'],"{'currency': 'GBP', 'initial': 1499, 'final': ...",[94],"[{'name': 'default', 'title': 'Buy Jagged Alli...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '2', 'description': 'Strategy'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '6 Jul, 2006'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/162...,"{'ids': [], 'notes': None}"
356,Jagged Alliance 2 Gold,1620,0.0,False,<p>The small country of Arulco has been taken ...,<p>The small country of Arulco has been taken ...,The small country of Arulco has been taken ove...,English,https:

In [154]:
steam_data[steam_data.duplicated()]

,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors
31,SiN Episodes: Emergence,1300,0.0,False,"You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","English, Russian, French",https://steamcdn-a.akamaihd.net/steam/apps/130...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,[],[],['Ritual Entertainment'],['Ritual Entertainment'],"{'currency': 'GBP', 'initial': 719, 'final': 7...",[70],"[{'name': 'default', 'title': 'Buy SiN Episode...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '10 May, 2006'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/130...,"{'ids': [], 'notes': None}"
32,SiN Episodes: Emergence,1300,0.0,False,"You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","English, Russian, French",https://steamcdn-a.akamaihd.net/steam/apps/130...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,[],[],['Ritual Entertainment'],['Ritual Entertainment'],"{'currency': 'GBP', 'initial': 719, 'final': 7...",[70],"[{'name': 'default', 'title': 'Buy SiN Episode...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '10 May, 2006'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/130...,"{'ids': [], 'notes': None}"
356,Jagged Alliance 2 Gold,1620,0.0,False,<p>The small country of Arulco has been taken ...,<p>The small country of Arulco has been taken ...,The small country of Arulco has been taken ove...,English,https://steamcdn-a.akamaihd.net/steam/apps/162...,http://www.jaggedalliance2.com/,{'minimum': '<p><strong>Minimum Configuration:...,[],[],['Strategy First'],['Strategy First'],"{'currency': 'GBP', 'initial': 1499, 'final': ...",[94],"[{'name': 'default', 'title': 'Buy Jagged Alli...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '2', 'description': 'Strategy'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '6 Jul, 2006'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/162...,"{'ids': [], 'notes': None}"
493,Crazy Machines 1.5,18430,0.0,False,Create Your Own Zany &quot;Rube Goldberg&quot;...,Create Your Own Zany &quot;Rube Goldberg&quot;...,Create Your Own Zany &quot;Rube Goldberg&quot;...,English,https://steamcdn-a.akamaihd.net/steam/apps/184...,NaN,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],[],['Fakt Software'],['Viva Media'],"{'currency': 'GBP', 'initial': 699, 'final': 6...","[1242, 58401]","[{'name': 'default', 'title': 'Buy Crazy Machi...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '12 Dec, 2008'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/184...,"{'ids': [], 'notes': None}"
494,Crazy Machines 1.5,18430,0.0,False,Create Your Own Zany &quot;Rube Goldberg&quot;...,Create Your Own Zany &quot;Rube Goldberg&quot;...,Create Your Own Zany &quot;Rube Goldberg&quot;...,English,https://steamcdn-a.akamaihd.net/st

In [91]:
steam_data.isnull().sum()

type                       0
name                       1
steam_appid                0
required_age               0
is_free                    0
detailed_description      26
about_the_game            26
short_description         26
supported_languages       14
header_image               0
website                 9834
pc_requirements            0
mac_requirements           0
linux_requirements         0
developers               115
publishers                 0
price_overview          3563
packages                3221
package_groups             0
platforms                  0
categories               565
genres                    47
screenshots               28
movies                  1929
achievements            2232
release_date               0
support_info               0
background                28
content_descriptors        0
dtype: int64

In [95]:
steam_data[steam_data['price_overview'].notnull()]

,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors
0,game,Counter-Strike,10,0.0,False,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,"English<strong>*</strong>, French<strong>*</st...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'GBP', 'initial': 719, 'final': 7...",[7],"[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '1 Nov, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
1,game,Team Fortress Classic,20,0.0,False,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'GBP', 'initial': 399, 'final': 3...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '1 Apr, 1999'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
2,game,Day of Defeat,30,0.0,False,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,"English, French, German, Italian, Spanish - Spain",https://steamcdn-a.akamaihd.net/steam/apps/30/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'GBP', 'initial': 399, 'final': 3...",[30],"[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '1 May, 2003'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/30/...,"{'ids': [], 'notes': None}"
3,game,Deathmatch Classic,40,0.0,False,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'GBP', 'initial': 399, 'final': 3...",[31],"[{'name': 'default', 'title': 'Buy Deathmatch ...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0

In [61]:
for data, col in zip(steam_data.iloc[0], steam_data.columns):
    print('-'*(20) + col + '-'*(30-len(col)) + '\n')
    print(data, '\n')

--------------------type--------------------------

game 

--------------------name--------------------------

Counter-Strike 

--------------------steam_appid-------------------

10 

--------------------required_age------------------

0.0 

--------------------is_free-----------------------

False 

--------------------detailed_description----------

Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role. 

--------------------about_the_game----------------

Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success

In [62]:
steam_data.columns.tolist()

['type',
 'name',
 'steam_appid',
 'required_age',
 'is_free',
 'detailed_description',
 'about_the_game',
 'short_description',
 'supported_languages',
 'header_image',
 'website',
 'pc_requirements',
 'mac_requirements',
 'linux_requirements',
 'developers',
 'publishers',
 'price_overview',
 'packages',
 'package_groups',
 'platforms',
 'categories',
 'genres',
 'screenshots',
 'movies',
 'achievements',
 'release_date',
 'support_info',
 'background',
 'content_descriptors']

In [ ]:
keep_columns = [
    'type',
    'name',
    'steam_appid',
    'required_age',
    'is_free',
    'detailed_description',
    'about_the_game',
    'short_description',
    'supported_languages',
    'header_image',
    'website',
    'pc_requirements',
    'mac_requirements',
    'linux_requirements',
    'developers',
    'publishers',
    'price_overview',
    'packages',
    'package_groups',
    'platforms',
    'categories',
    'genres',
    'movies',
    'achievements',
    'release_date',
    'support_info',
    'content_descriptors'
]

steam_data[keep_columns].head()

In [24]:
null_counts = raw_steam_data.isnull().sum()
columns = raw_steam_data.columns[null_counts > 20000]

display(null_counts)
raw_steam_data[columns].head()

type                         149
name                           1
steam_appid                    0
required_age                 149
is_free                      149
controller_support         23237
dlc                        24260
detailed_description         175
about_the_game               175
short_description            175
fullgame                   29235
supported_languages          163
header_image                 149
website                     9983
pc_requirements              149
mac_requirements             149
linux_requirements           149
legal_notice               19168
drm_notice                 29077
ext_user_account_notice    28723
developers                   264
publishers                   149
demos                      27096
price_overview              3712
packages                    3370
package_groups               149
platforms                    149
metacritic                 26254
reviews                    23330
categories                   714
genres    

,controller_support,dlc,fullgame,drm_notice,ext_user_account_notice,demos,metacritic,reviews,recommendations
0,NaN,NaN,NaN,NaN,NaN,NaN,"{'score': 88, 'url': 'https://www.metacritic.c...",NaN,{'total': 65735}
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'total': 2802}
2,NaN,NaN,NaN,NaN,NaN,NaN,"{'score': 79, 'url': 'https://www.metacritic.c...",NaN,{'total': 1992}
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'total': 931}
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'total': 4355}


In [36]:
steam_data = raw_steam_data[raw_steam_data['type'].notnull()]
steam_data.head()

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,...,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,game,Counter-Strike,10,0.0,False,NaN,NaN,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,...,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 65735},{'total': 0},"{'coming_soon': False, 'date': '1 Nov, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
1,game,Team Fortress Classic,20,0.0,False,NaN,NaN,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,...,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 2802},{'total': 0},"{'coming_soon': False, 'date': '1 Apr, 1999'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
2,game,Day of Defeat,30,0.0,False,NaN,NaN,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,...,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 1992},{'total': 0},"{'coming_soon': False, 'date': '1 May, 2003'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/30/...,"{'ids': [], 'notes': None}"
3,game,Deathmatch Classic,40,0.0,False,NaN,NaN,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,...,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 931},{'total': 0},"{'coming_soon': False, 'date': '1 Jun, 2001'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/40/...,"{'ids': [], 'notes': None}"
4,game,Half-Life: Opposing Force,50,0.0,False,NaN,NaN,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,...,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 4355},{'total': 0},"{'coming_soon': False, 'date': '1 Nov, 1999'}","{'url': 'https://help.steampowered.com', 'emai...",https://steamcdn-a.akamaihd.net/steam/apps/50/...,"{'ids': [], 'notes': None}"


In [77]:
steam_data.columns

Index(['type', 'name', 'steam_appid', 'required_age', 'is_free',
       'detailed_description', 'about_the_game', 'short_description',
       'supported_languages', 'header_image', 'website', 'pc_requirements',
       'mac_requirements', 'linux_requirements', 'developers', 'publishers',
       'price_overview', 'packages', 'package_groups', 'platforms',
       'categories', 'genres', 'screenshots', 'movies', 'achievements',
       'release_date', 'support_info', 'background', 'content_descriptors'],
      dtype='object')

In [79]:
steam_data['publishers'].value_counts()

['']                                            234
['Big Fish Games']                              231
['Strategy First']                              139
['Ubisoft']                                     117
['THQ Nordic']                                  106
['Square Enix']                                 105
['Sekai Project']                                98
['Choice of Games']                              95
['1C Entertainment']                             91
['SEGA']                                         91
['Dagestan Technology']                          90
['Degica']                                       87
['KISS ltd']                                     78
['Devolver Digital']                             73
['Plug In Digital']                              73
['AGM PLAYISM']                                  71
['KOEI TECMO GAMES CO., LTD.']                   71
['Slitherine Ltd.']                              71
['Artifex Mundi']                                68
['Buka Enter

In [82]:
steam_data[steam_data['type'].isnull()]

,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors
26,NaN,Half-Life: Opposing Force,852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,NaN,Half-Life: Opposing Force,4330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256,NaN,Half-Life: Opposing Force,8740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
264,NaN,Half-Life: Opposing Force,8955,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
336,NaN,Half-Life: Opposing Force,11610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
376,NaN,Half-Life: Opposing Force,12650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
377,NaN,Half-Life: Opposing Force,12660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387,NaN,Half-Life: Opposing Force,13120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
394,NaN,Half-Life: Opposing Force,13260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
419,NaN,Half-Life: Opposing Force,15350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Steamspy

In [6]:
raw_steamspy_data = pd.read_csv('../data/steamspy_data.csv')

print('Rows:', raw_steamspy_data.shape[0])
print('Columns:', raw_steamspy_data.shape[1])
raw_steamspy_data.head()

Rows: 29235
Columns: 20


,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,10,Counter-Strike,Valve,Valve,NaN,124534,3339,0,"10,000,000 .. 20,000,000",17612,709,317,26,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,14923,"{'Action': 2681, 'FPS': 2048, 'Multiplayer': 1..."
1,20,Team Fortress Classic,Valve,Valve,NaN,3318,633,0,"5,000,000 .. 10,000,000",277,15,62,15,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,87,"{'Action': 208, 'FPS': 188, 'Multiplayer': 172..."
2,30,Day of Defeat,Valve,Valve,NaN,3416,398,0,"5,000,000 .. 10,000,000",187,0,34,0,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Spain",Action,130,"{'FPS': 138, 'World War II': 122, 'Multiplayer..."
3,40,Deathmatch Classic,Valve,Valve,NaN,1273,267,0,"5,000,000 .. 10,000,000",258,0,184,0,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,4,"{'Action': 85, 'FPS': 71, 'Multiplayer': 58, '..."
4,50,Half-Life: Opposing Force,Gearbox Software,Valve,NaN,5250,288,0,"5,000,000 .. 10,000,000",624,0,415,0,499.0,499.0,0.0,"English, French, German, Korean",Action,71,"{'FPS': 235, 'Action': 211, 'Sci-fi': 166, 'Si..."


In [11]:
raw_steamspy_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29235 entries, 0 to 29234
Data columns (total 20 columns):
appid              29235 non-null int64
name               29230 non-null object
developer          29038 non-null object
publisher          28955 non-null object
score_rank         58 non-null float64
positive           29235 non-null int64
negative           29235 non-null int64
userscore          29235 non-null int64
owners             29235 non-null object
average_forever    29235 non-null int64
average_2weeks     29235 non-null int64
median_forever     29235 non-null int64
median_2weeks      29235 non-null int64
price              29206 non-null float64
initialprice       29213 non-null float64
discount           29213 non-null float64
languages          29141 non-null object
genre              29083 non-null object
ccu                29235 non-null int64
tags               29235 non-null object
dtypes: float64(4), int64(9), object(7)
memory usage: 4.5+ MB


In [2]:
raw_steamspy_data.isnull().sum()

appid                  0
name                   5
developer            197
publisher            280
score_rank         29177
positive               0
negative               0
userscore              0
owners                 0
average_forever        0
average_2weeks         0
median_forever         0
median_2weeks          0
price                 29
initialprice          22
discount              22
languages             94
genre                152
ccu                    0
tags                   0
dtype: int64

In [4]:
raw_steamspy_data['score_rank'].value_counts(dropna=False)

NaN       29177
 100.0       12
 97.0        11
 96.0        11
 95.0         7
 98.0         7
 94.0         6
 99.0         3
 93.0         1
Name: score_rank, dtype: int64

In [6]:
raw_steamspy_data[raw_steamspy_data['score_rank'].notnull()]

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
562,22490,Fallout: New Vegas,Obsidian Entertainment,Bethesda Softworks,100.0,46753,2153,95,"500,000 .. 1,000,000",2810,0,3491,0,999.0,999.0,0.0,"English, French, German, Italian, Spanish","Action, RPG",566,"{'Open World': 2453, 'RPG': 2327, 'Post-apocal..."
6257,371120,Discouraged Workers,YGGDRASIL STUDIO,YGGDRASIL STUDIO,95.0,61,38,59,"0 .. 20,000",0,0,0,0,1499.0,1499.0,0.0,"English, Korean, Spanish - Spain, Russian, Sim...","Adventure, Indie",0,"{'Nudity': 100, 'Sexual Content': 69, 'Mature'..."
9570,460870,GOD EATER RESURRECTION,"BANDAI NAMCO Studio, SHIFT, QLOC",BANDAI NAMCO Entertainment,97.0,441,141,82,"0 .. 20,000",1154,1,1904,1,0.0,0.0,0.0,NaN,NaN,43,"{'Anime': 28, 'Action': 24, 'RPG': 18, 'JRPG':..."
10906,502300,Heartomics: Lost Count,Heartomics,Heartomics,95.0,27,18,51,"0 .. 20,000",0,0,0,0,999.0,999.0,0.0,English,"Sexual Content, Nudity, Action, Indie",1,"{'Bullet Hell': 89, ""Shoot 'Em Up"": 82, 'Sexua..."
11865,526370,SUPER KINKY,Dark Illusions Entertainment LLC,Dark Illusions Entertainment LLC,96.0,43,18,70,"0 .. 20,000",46,0,46,0,499.0,499.0,0.0,English,"Action, Indie",0,"{'Sexual Content': 41, 'Gore': 40, 'Action': 4..."
12571,545250,Bloody Boobs,GK,Dagestan Technology,94.0,520,521,49,"20,000 .. 50,000",271,0,271,0,399.0,399.0,0.0,"English, Russian","Action, Adventure, Indie",2,"{'Sexual Content': 123, 'Nudity': 81, 'Female ..."
12990,555310,Satellite,"7DOTS, Rock Frog","7DOTS, Angry games, RuVN",97.0,44,9,82,"0 .. 20,000",0,0,0,0,299.0,299.0,0.0,"English, Russian","Adventure, Casual, Indie",0,"{'Adventure': 25, 'Sexual Content': 24, 'Indie..."
13167,560000,Ladykiller in a Bind,Love Conquers All Games,Love Conquers All Games,100.0,170,8,95,"0 .. 20,000",0,0,0,0,1999.0,1999.0,0.0,English,"Sexual Content, Nudity, Indie",4,"{'Sexual Content': 64, 'Nudity': 48, 'Visual N..."
13732,575050,Little Triangle,CottonGame,Dreamoji,100.0,330,16,98,"0 .. 20,000",0,0,0,0,1199.0,1199.0,0.0,"English, Simplified Chinese","Action, Adventure, Indie, Early Access",3,"{'Early Access': 30, 'Action': 27, 'Indie': 27..."
14294,588920,BADASS,Toothless T-Rex,大きくて強い,94.0,15,14,51,"0 .. 20,000",0,0,0,0,499.0,499.0,0.0,English,"Adventure, Casual, Indie, RPG, Simulation, Str...",0,"{'Nudity': 62, 'Sexual Content': 59, 'RPG': 36..."


In [10]:
raw_steamspy_data.dropna(axis=1, thresh=500)

,appid,name,developer,publisher,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,10,Counter-Strike,Valve,Valve,124534,3339,0,"10,000,000 .. 20,000,000",17612,709,317,26,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,14923,"{'Action': 2681, 'FPS': 2048, 'Multiplayer': 1..."
1,20,Team Fortress Classic,Valve,Valve,3318,633,0,"5,000,000 .. 10,000,000",277,15,62,15,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,87,"{'Action': 208, 'FPS': 188, 'Multiplayer': 172..."
2,30,Day of Defeat,Valve,Valve,3416,398,0,"5,000,000 .. 10,000,000",187,0,34,0,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Spain",Action,130,"{'FPS': 138, 'World War II': 122, 'Multiplayer..."
3,40,Deathmatch Classic,Valve,Valve,1273,267,0,"5,000,000 .. 10,000,000",258,0,184,0,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,4,"{'Action': 85, 'FPS': 71, 'Multiplayer': 58, '..."
4,50,Half-Life: Opposing Force,Gearbox Software,Valve,5250,288,0,"5,000,000 .. 10,000,000",624,0,415,0,499.0,499.0,0.0,"English, French, German, Korean",Action,71,"{'FPS': 235, 'Action': 211, 'Sci-fi': 166, 'Si..."
5,60,Ricochet,Valve,Valve,2758,684,0,"5,000,000 .. 10,000,000",175,0,10,0,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,8,"{'Action': 108, 'FPS': 71, 'Multiplayer': 59, ..."
6,70,Half-Life,Valve,Valve,27755,1100,0,"5,000,000 .. 10,000,000",1300,24,83,32,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,374,"{'FPS': 929, 'Classic': 874, 'Action': 766, 'S..."
7,80,Counter-Strike: Condition Zero,Valve,Valve,12120,1439,0,"10,000,000 .. 20,000,000",427,6,43,6,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,627,"{'Action': 377, 'FPS': 311, 'Multiplayer': 224..."
8,130,Half-Life: Blue Shift,Gearbox Software,Valve,3822,420,0,"5,000,000 .. 10,000,000",361,0,205,0,499.0,499.0,0.0,"English, French, German",Action,34,"{'FPS': 213, 'Action': 187, 'Sci-fi': 144, 'Si..."
9,220,Half-Life 2,Valve,Valve,67902,2419,0,"10,000,000 .. 20,000,000",691,19,402,19,999.0,999.0,0.0,"English, French, German, Italian, Korean, Span...",Action,628,"{'FPS': 2056, 'Action': 1761, 'Sci-fi': 1589, ..."


In [12]:
raw_steamspy_data[raw_steamspy_data['developer'].isnull()]

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
26,852,ValveTestApp852,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
147,4330,Star Trek: DAC - Demo,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
256,8740,Puzzlegeddon,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,1,[]
264,8955,Borderlands DLC: Claptrap's New Robot Revolution,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
285,9730,Tycoon City: New York,NaN,Retroism,NaN,58,102,0,"20,000 .. 50,000",0,0,0,0,999.0,999.0,0.0,English,NaN,0,"{'Simulation': 34, 'Management': 18, 'City Bui..."
330,11370,Nikopol: Secrets of the Immortals,NaN,Meridian4,NaN,163,130,0,"100,000 .. 200,000",0,0,0,0,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Spain","Adventure, Indie",0,"{'Adventure': 61, 'Point & Click': 43, 'Indie'..."
331,11390,Crash Time 2,NaN,Meridian4,NaN,1148,1017,0,"500,000 .. 1,000,000",30,0,37,0,999.0,999.0,0.0,"English, French, German","Action, Racing",2,"{'Racing': 102, 'Action': 63, 'Open World': 57..."
379,12690,Hunting Unlimited 2010,NaN,"ValuSoft, Retroism",NaN,142,39,0,"50,000 .. 100,000",0,0,0,0,999.0,999.0,0.0,English,"Simulation, Sports",9,"{'Simulation': 35, 'Hunting': 30, 'Sports': 20..."
387,13120,America's Army 3 Beta,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
394,13260,Unreal Development Kit,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]


In [13]:
raw_steamspy_data.dropna()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
562,22490,Fallout: New Vegas,Obsidian Entertainment,Bethesda Softworks,100.0,46753,2153,95,"500,000 .. 1,000,000",2810,0,3491,0,999.0,999.0,0.0,"English, French, German, Italian, Spanish","Action, RPG",566,"{'Open World': 2453, 'RPG': 2327, 'Post-apocal..."
6257,371120,Discouraged Workers,YGGDRASIL STUDIO,YGGDRASIL STUDIO,95.0,61,38,59,"0 .. 20,000",0,0,0,0,1499.0,1499.0,0.0,"English, Korean, Spanish - Spain, Russian, Sim...","Adventure, Indie",0,"{'Nudity': 100, 'Sexual Content': 69, 'Mature'..."
10906,502300,Heartomics: Lost Count,Heartomics,Heartomics,95.0,27,18,51,"0 .. 20,000",0,0,0,0,999.0,999.0,0.0,English,"Sexual Content, Nudity, Action, Indie",1,"{'Bullet Hell': 89, ""Shoot 'Em Up"": 82, 'Sexua..."
11865,526370,SUPER KINKY,Dark Illusions Entertainment LLC,Dark Illusions Entertainment LLC,96.0,43,18,70,"0 .. 20,000",46,0,46,0,499.0,499.0,0.0,English,"Action, Indie",0,"{'Sexual Content': 41, 'Gore': 40, 'Action': 4..."
12571,545250,Bloody Boobs,GK,Dagestan Technology,94.0,520,521,49,"20,000 .. 50,000",271,0,271,0,399.0,399.0,0.0,"English, Russian","Action, Adventure, Indie",2,"{'Sexual Content': 123, 'Nudity': 81, 'Female ..."
12990,555310,Satellite,"7DOTS, Rock Frog","7DOTS, Angry games, RuVN",97.0,44,9,82,"0 .. 20,000",0,0,0,0,299.0,299.0,0.0,"English, Russian","Adventure, Casual, Indie",0,"{'Adventure': 25, 'Sexual Content': 24, 'Indie..."
13167,560000,Ladykiller in a Bind,Love Conquers All Games,Love Conquers All Games,100.0,170,8,95,"0 .. 20,000",0,0,0,0,1999.0,1999.0,0.0,English,"Sexual Content, Nudity, Indie",4,"{'Sexual Content': 64, 'Nudity': 48, 'Visual N..."
13732,575050,Little Triangle,CottonGame,Dreamoji,100.0,330,16,98,"0 .. 20,000",0,0,0,0,1199.0,1199.0,0.0,"English, Simplified Chinese","Action, Adventure, Indie, Early Access",3,"{'Early Access': 30, 'Action': 27, 'Indie': 27..."
14294,588920,BADASS,Toothless T-Rex,大きくて強い,94.0,15,14,51,"0 .. 20,000",0,0,0,0,499.0,499.0,0.0,English,"Adventure, Casual, Indie, RPG, Simulation, Str...",0,"{'Nudity': 62, 'Sexual Content': 59, 'RPG': 36..."
14435,592750,SPACE-FRIGHT,R I MAD,R I MAD,98.0,25,3,88,"0 .. 20,000",0,0,0,0,699.0,699.0,0.0,English,"Sexual Content, Violent, Gore, Adventure, Indie",0,"{'Gore': 22, 'Sexual Content': 22, 'Indie': 21..."


In [25]:
import time
for i in range(5):
    print('\r{}'.format(i), end='')
    time.sleep(0.5)
    
print('\rdone')

done
